In [ ]:
# Step 1: Install Required Libraries
# First, make sure you have the necessary libraries installed:

!pip install transformers datasets torch

In [ ]:
# Step 2: Prepare Your Data
# Create a CSV file for your training data with columns: ticket_id, ticket_subject, ticket_body, type, department, language, priority.

In [ ]:
# Step 3: Load and Preprocess Data
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Load your training data
train_data = pd.read_csv('training_data.csv')

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['ticket_subject'] + " " + examples['ticket_body'], padding="max_length", truncation=True)

tokenized_datasets = train_dataset.map(tokenize_function, batched=True)

In [ ]:
# Step 4: Fine-Tune the Model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Define the model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# Train the model
trainer.train()

In [ ]:
# Step 5: Evaluate and Predict
# Load your test data
test_data = pd.read_csv('test_data.csv')
test_dataset = Dataset.from_pandas(test_data)

# Tokenize the test data
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)

# Predict
predictions = trainer.predict(tokenized_test_datasets)
predicted_labels = predictions.predictions.argmax(-1)

# Map predictions to labels
label_map = {0: 'Incident', 1: 'Request', 2: 'Problem', 3: 'Change'}
predicted_labels = [label_map[label] for label in predicted_labels]

# Add predictions to the test data
test_data['predicted_type'] = predicted_labels

In [ ]:
# Step 6: Save the Results
test_data.to_csv('predicted_test_data.csv', index=False)